In [185]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
print("Import libraries done.")

Import libraries done.


In [186]:
df_original = pd.read_csv(r"/Users/mexmex/Documents/3-Math_LU/HT2025/STAN48-Programming-for-Data-Science/Final Project/Data/cleaned_credit_risk_dataset.csv")
df_original.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,person_home_ownership_OWN,person_home_ownership_OTHER,loan_intent_MEDICAL,loan_intent_VENTURE,loan_intent_PERSONAL,loan_intent_HOMEIMPROVEMENT,loan_intent_DEBTCONSOLIDATION
0,21,9600,5.0,1,1000,11.14,0,0.10,0,2,0,1,0,0,0,0,0,0
1,25,9600,1.0,2,5500,12.87,1,0.57,0,3,1,0,0,1,0,0,0,0
2,23,65500,4.0,2,35000,15.23,1,0.53,0,2,0,0,0,1,0,0,0,0
3,24,54400,8.0,2,35000,14.27,1,0.55,1,4,0,0,0,1,0,0,0,0
4,21,9900,2.0,0,2500,7.14,1,0.25,0,2,0,1,0,0,1,0,0,0


In [187]:
df = df_original.copy()
df.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,person_home_ownership_OWN,person_home_ownership_OTHER,loan_intent_MEDICAL,loan_intent_VENTURE,loan_intent_PERSONAL,loan_intent_HOMEIMPROVEMENT,loan_intent_DEBTCONSOLIDATION
0,21,9600,5.0,1,1000,11.14,0,0.10,0,2,0,1,0,0,0,0,0,0
1,25,9600,1.0,2,5500,12.87,1,0.57,0,3,1,0,0,1,0,0,0,0
2,23,65500,4.0,2,35000,15.23,1,0.53,0,2,0,0,0,1,0,0,0,0
3,24,54400,8.0,2,35000,14.27,1,0.55,1,4,0,0,0,1,0,0,0,0
4,21,9900,2.0,0,2500,7.14,1,0.25,0,2,0,1,0,0,1,0,0,0


In [188]:
df['loan_status'].value_counts()

loan_status
0    25321
1     7088
Name: count, dtype: int64

In [189]:
class LogisticRegression:
    def __init__(self, Y_resp, X_data):
        self.X = np.c_[np.ones(len(X_data)), (X_data - np.mean(X_data, axis=0)) / np.std(X_data, axis=0)]
        self.Y = Y_resp

    def loglik(self, betas):
        return betas @ self.X.T @ self.Y - np.sum(np.log(1 + np.exp(self.X @ betas)))

    def grad_loglik(self, betas):
       
        #print("X matrix =", self.X)
        return self.X.T @ (self.Y - 1 / (1 + np.exp(-self.X @ betas)))

    def hess_loglik(self, betas):
       
        p = 1 / (1 + np.exp(-self.X @ betas))
        W = np.diag(p * (1 - p))
        return -self.X.T @ W @ self.X  - 1e-6 * np.eye(self.X.shape[1])
    
    def newton_raphson(self, tol=1e-6, max_iter=100):
        """ Use this method to find zeros of the gradient of the log-likelihood function.
        
        Algorithm:
        betas_new = betas - f(betas) / f'(betas)
        where f is the gradient of the log-likelihood function and f' is the Hessian of the log-likelihood function.

        So:
        betas_new = betas - grad_loglik(betas) @ inv(hess_loglik(betas))
        Use np.linalg.solve instead of np.linalg.inv for better numerical stability. And it's expensive to compute the inverse of a matrix.

        Update betas until convergence or max_iter is reached.
        """
        betas = np.zeros(self.X.shape[1])  # Initialize betas to zero vector
        for i in range(max_iter):
            print(f"Iteration {i+1}, betas = {betas}")
            # f_betas is the gradient of the log-likelihood function at betas
            f_betas = self.grad_loglik(betas)

            # f_grad_betas is the Hessian of the log-likelihood function at betas
            f_grad_betas = self.hess_loglik(betas)
            
            betas_new = betas - np.linalg.solve(f_grad_betas, f_betas)

            if np.linalg.norm(betas_new - betas) < tol:
                print(f"Converged in {i+1} iterations.")
                return betas_new

            betas = betas_new
        print("Max iterations reached.")
        return betas
    
    def make_predictions(self,X_test, betas, threshold=0.5):
        """ Make predictions based on the fitted model.
        If the predicted probability is greater than the threshold, predict 1 (fully paid), else 0 (charged off).
        """
        X_test = np.c_[np.ones(len(X_test)), (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)]
        probs = 1 / (1 + np.exp(-X_test @ betas))
        return (probs >= threshold).astype(int)
    
    def bic(self, betas):
        return np.log(self.X.shape[0]) * (self.X.shape[1]) - 2 * self.loglik(betas)
    
    def __str__(self):
        return f"LogisticRegression with {self.X.shape[1]-1} predictors and {self.X.shape[0]} observations."    

print("LogisticRegression class defined.")

LogisticRegression class defined.


In [190]:
train_proportion = round(0.8 * df.shape[0])
train_proportion

25927

In [191]:
X_train = df.loc[:train_proportion-1, df.columns != 'loan_status'].values
y_train = df['loan_status'].loc[:train_proportion-1].values

X_test = df.loc[train_proportion:, df.columns != 'loan_status'].values
y_test = df['loan_status'].loc[train_proportion:].values
X_train.shape, y_train.shape, X_test.shape, y_test.shape


((25927, 17), (25927,), (6482, 17), (6482,))

In [192]:
model0 = LogisticRegression(Y_resp=y_train, X_data=X_train)
print(model0)

LogisticRegression with 17 predictors and 25927 observations.


In [193]:
model0_betas = model0.newton_raphson()

Iteration 1, betas = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Iteration 2, betas = [-1.10579705 -0.03475821  0.12685454 -0.00292703  0.8212138  -0.45290941
 -0.27673608  0.8657347  -0.03397831 -0.01625988 -0.18265347 -0.23808605
 -0.00932168  0.12801685 -0.00732255  0.05839991  0.15958625  0.14301479]
Iteration 3, betas = [-1.57413335 -0.05120097  0.1051194  -0.01042632  1.12103907 -0.62819466
 -0.275218    1.24460783 -0.04250844 -0.02389187 -0.31471685 -0.45533053
 -0.02140561  0.22107989 -0.02469411  0.10107301  0.27586246  0.24634281]
Iteration 4, betas = [-1.81386465 -0.05713317  0.06822458 -0.0173737   1.23233002 -0.69699357
 -0.23323398  1.41336789 -0.0467461  -0.02622934 -0.38570243 -0.60521118
 -0.02846204  0.26783296 -0.04046221  0.12182187  0.33693062  0.29970111]
Iteration 5, betas = [-1.86730667 -0.05799793  0.04875253 -0.01922414  1.25277659 -0.70247173
 -0.22251523  1.43980411 -0.04777469 -0.02646992 -0.39982568 -0.64680186
 -0.02989123  0.27640025 -0.04478

In [194]:
class ConfusionMatrix:
    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred
        self.tp = np.sum((y_true == 1) & (y_pred == 1))
        self.tn = np.sum((y_true == 0) & (y_pred == 0))
        self.fp = np.sum((y_true == 0) & (y_pred == 1))
        self.fn = np.sum((y_true == 1) & (y_pred == 0))
        self.accuracy = (self.tp + self.tn) / len(y_true)
        self.precision = self.tp / (self.tp + self.fp) if (self.tp + self.fp) > 0 else 0
        self.recall = self.tp / (self.tp + self.fn) if (self.tp + self.fn) > 0 else 0
        self.f1_score = 2 * (self.precision * self.recall) / (self.precision + self.recall) if (self.precision + self.recall) > 0 else 0

    def __str__(self):
        return f"Confusion Matrix:\nTP: {self.tp}, TN: {self.tn}, FP: {self.fp}, FN: {self.fn}\nAccuracy: {self.accuracy:.4f}, Precision: {self.precision:.4f}, Recall: {self.recall:.4f}, F1 Score: {self.f1_score:.4f}"


In [195]:
model0_p_pred = model0.make_predictions(X_test=X_test, betas=model0_betas, threshold=0.5)
model0_p_pred.shape

(6482,)

In [196]:
print(ConfusionMatrix(y_true=y_test, y_pred=model0_p_pred))

Confusion Matrix:
TP: 731, TN: 4848, FP: 342, FN: 561
Accuracy: 0.8607, Precision: 0.6813, Recall: 0.5658, F1 Score: 0.6182


In [197]:
class LinearRegression():
    def __init__(self, Y_resp, X_data):
        self.X = np.c_[np.ones(len(X_data)), (X_data - np.mean(X_data, axis=0)) / np.std(X_data, axis=0)]
        self.Y = Y_resp
    
    def betas_estimates(self):
        return np.linalg.solve(self.X.T @ self.X, self.X.T @ self.Y)
    
    def Y_hat(self, betas_estimates):
        return self.X @ betas_estimates
    
print("LinearRegression class defined.")

LinearRegression class defined.


In [198]:
X_data = df.loc[:, df.columns != 'loan_status']
X_data.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,person_home_ownership_OWN,person_home_ownership_OTHER,loan_intent_MEDICAL,loan_intent_VENTURE,loan_intent_PERSONAL,loan_intent_HOMEIMPROVEMENT,loan_intent_DEBTCONSOLIDATION
0,21,9600,5.0,1,1000,11.14,0.10,0,2,0,1,0,0,0,0,0,0
1,25,9600,1.0,2,5500,12.87,0.57,0,3,1,0,0,1,0,0,0,0
2,23,65500,4.0,2,35000,15.23,0.53,0,2,0,0,0,1,0,0,0,0
3,24,54400,8.0,2,35000,14.27,0.55,1,4,0,0,0,1,0,0,0,0
4,21,9900,2.0,0,2500,7.14,0.25,0,2,0,1,0,0,1,0,0,0


In [199]:
def vif(X_data):
    X_columns = X_data.columns
    X_data = X_data.values
    vif_dict = {}
    for i in range(X_data.shape[1]):
        model_X_j =  LinearRegression(Y_resp=X_data[:,i], X_data=np.delete(X_data,i, axis=1))
        betas = model_X_j.betas_estimates()
        X_j_hat =model_X_j.Y_hat(betas)
        #R_squared = (np.linalg.norm(X_j_hat) / np.linalg.norm(X_data[:,i]))**2
        residuals = X_data[:,i] - X_j_hat
        R_squared = 1 - (np.sum(residuals**2) / np.sum((X_data[:,i] - np.mean(X_data[:,i]))**2))

        vif_j = 1/(1-R_squared)
        vif_dict[X_columns[i]] = vif_j
    
    return vif_dict



In [200]:
vif0 = vif(X_data=X_data)
vif_df = pd.DataFrame(list(vif0.items()), columns=["Variable", "VIF"]).sort_values(by='VIF',ascending=False)
vif_df
# loan_grade is the most problematic variable
# exclude

,Variable,VIF
3,loan_grade,8.734323
5,loan_int_rate,8.362199
0,person_age,4.442829
8,cb_person_cred_hist_length,4.379426
4,loan_amnt,2.559309
6,loan_percent_income,2.484904
1,person_income,1.805183
12,loan_intent_MEDICAL,1.592588
13,loan_intent_VENTURE,1.566066
14,loan_intent_PERSONAL,1.552921


In [201]:
X_train1 = df.loc[:train_proportion-1].drop(columns=['loan_status', 'loan_grade']).values
X_test1 = df.loc[train_proportion:].drop(columns=['loan_status', 'loan_int_rate']).values
X_train1.shape,y_train.shape, X_test1.shape, y_test.shape


model1 = LogisticRegression(Y_resp=y_train,X_data=X_train1)
print(model1)

LogisticRegression with 16 predictors and 25927 observations.


In [202]:
model1_betas = model1.newton_raphson()

Iteration 1, betas = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Iteration 2, betas = [-1.10579705 -0.02979102  0.12530394 -0.00465914 -0.44716626  0.45483763
  0.87266572  0.03961813 -0.02414683 -0.17614721 -0.23763339 -0.00980807
  0.12679761 -0.00665422  0.05673004  0.16508977  0.14365474]
Iteration 3, betas = [-1.57163375 -0.03960309  0.10525696 -0.01797582 -0.60242896  0.77797172
  1.22895738  0.03384957 -0.03796372 -0.30154981 -0.44836479 -0.01871996
  0.21266127 -0.02364892  0.09433938  0.27623896  0.24002106]
Iteration 4, betas = [-1.81200132 -0.04121434  0.07577809 -0.02812908 -0.66494881  0.94528405
  1.38940898  0.02881574 -0.04380136 -0.36715401 -0.58781768 -0.02407784
  0.2542437  -0.03814247  0.11146872  0.33132708  0.28748471]
Iteration 5, betas = [-1.86385692 -0.04115023  0.06126529 -0.03050123 -0.67160088  0.97811999
  1.41602857  0.02795639 -0.04479527 -0.37986584 -0.62463703 -0.0252145
  0.26176431 -0.04189028  0.11437104  0.34208     0.29652184]
Iteration 6

In [203]:
model1_p_pred = model1.make_predictions(X_test=X_test1, betas=model1_betas, threshold=0.5)
model1_p_pred.shape

(6482,)

In [204]:
print(ConfusionMatrix(y_true=y_test, y_pred=model1_p_pred))

Confusion Matrix:
TP: 528, TN: 4440, FP: 750, FN: 764
Accuracy: 0.7664, Precision: 0.4131, Recall: 0.4087, F1 Score: 0.4109


In [205]:
X_data_excl = df.loc[:train_proportion-1].drop(columns=['loan_status', 'loan_grade'])
main_effects = list(X_data_excl.columns)
main_effects

['person_age',
 'person_income',
 'person_emp_length',
 'loan_amnt',
 'loan_int_rate',
 'loan_percent_income',
 'cb_person_default_on_file',
 'cb_person_cred_hist_length',
 'person_home_ownership_MORTGAGE',
 'person_home_ownership_OWN',
 'person_home_ownership_OTHER',
 'loan_intent_MEDICAL',
 'loan_intent_VENTURE',
 'loan_intent_PERSONAL',
 'loan_intent_HOMEIMPROVEMENT',
 'loan_intent_DEBTCONSOLIDATION']

In [206]:
selectec_predictors = []
current_bic = np.inf
X_current = np.ones((X_data_excl.values.shape[0], 1))
null_model = LogisticRegression(Y_resp=y_train,X_data=X_current)
null_model_betas = null_model.newton_raphson()
current_bic = null_model.bic(null_model_betas)
print(f"Initial BIC (null model): {current_bic:.3f}")

Iteration 1, betas = [0. 0.]


/var/folders/w0/m62gyqfs4fz6fxrxl6443x3c0000gn/T/ipykernel_31530/2473851690.py:3: RuntimeWarning: invalid value encountered in divide
  self.X = np.c_[np.ones(len(X_data)), (X_data - np.mean(X_data, axis=0)) / np.std(X_data, axis=0)]


Iteration 2, betas = [nan nan]
Iteration 3, betas = [nan nan]
Iteration 4, betas = [nan nan]
Iteration 5, betas = [nan nan]
Iteration 6, betas = [nan nan]
Iteration 7, betas = [nan nan]
Iteration 8, betas = [nan nan]
Iteration 9, betas = [nan nan]
Iteration 10, betas = [nan nan]
Iteration 11, betas = [nan nan]
Iteration 12, betas = [nan nan]
Iteration 13, betas = [nan nan]
Iteration 14, betas = [nan nan]
Iteration 15, betas = [nan nan]
Iteration 16, betas = [nan nan]
Iteration 17, betas = [nan nan]
Iteration 18, betas = [nan nan]
Iteration 19, betas = [nan nan]
Iteration 20, betas = [nan nan]
Iteration 21, betas = [nan nan]
Iteration 22, betas = [nan nan]
Iteration 23, betas = [nan nan]
Iteration 24, betas = [nan nan]
Iteration 25, betas = [nan nan]
Iteration 26, betas = [nan nan]
Iteration 27, betas = [nan nan]
Iteration 28, betas = [nan nan]


KeyboardInterrupt: 

In [159]:
np.ones(X_data_excl.values.shape[0])

array([1., 1., 1., ..., 1., 1., 1.], shape=(25927,))